In [1]:
import pandas as pd
from IPython.core.display import display, HTML

all_bids_ids = pd.read_csv("nfb_subset_SUBJECT_IDS.csv")
all_bids_ids['ID'] = all_bids_ids['ID'].str[4:]
all_bids_ids['conn_ID'] = all_bids_ids.index + 1

all_score_ids = pd.read_csv("events_IDs.csv")

bids_subset_ids = pd.merge(all_score_ids, all_bids_ids, how='inner', on='ID').sort_values(by=['ID']).reset_index(drop=True)
display(bids_subset_ids)

,ID,conn_ID
0,A00028185,1
1,A00032875,3
2,A00033747,4
3,A00034854,5
4,A00035072,6
...,...,...
133,A00066827,151
134,A00066926,152
135,A00072203,153
136,A00073600,154


In [2]:
"""

.. gc-fmri

================================
Granger 'causality' of fMRI data
================================

Granger 'causality' analysis provides an asymmetric measure of the coupling
between two time-series. When discussing this analysis method, we will put the
word 'causality' in single quotes, as we believe that use of this word outside
of quotes should be reserved for particular circumstances, often not fulfilled
in the analysis of simultaneously recorder neuroscientific time-series (see
[Pearl2009]_ for an extensive discussion of this distinction).

The central idea behind this analysis is that time-series can be described in
terms of a time-delayed auto-regressive model of the form:

.. math::

   x_t = \sum_{i=1}^{n}a_i x_{t-i} + \epsilon_t

Here, the past behaviour of a single time-series is used in order to predict
the current value of the time-series. In Granger 'causality' analysis, we test
whether the addition of a prediction of the time-series from another
time-series through a multivariate auto-regressive model may improve our
prediction of the present behavior of the time-series (reducing the value of
the error term $\epsilon_t$):

.. math::

   x_t = \sum_{i=1}^{n}(a_i x_{t-i} + b_i y_{t-i}) + \epsilon_t


In our implementation of the algorithms used for this analysis, we follow
closely the description put forth by Ding et al. ([Ding2006]_). Also, see
:ref:`mar` and :ref:`ar` for examples even more closely modeled on the
examples mentioned in their paper.

Here, we will demonstrate the use of Granger 'causality' analysis with fMRI
data. The data is provided as part of the distribution and is taken from a
'resting state' scan. The data was motion corrected and averaged from several
ROIs.

We start by importing the needed modules:

"""

import os

import numpy as np
import matplotlib.pyplot as plt

import nitime
import nitime.analysis as nta
import nitime.timeseries as ts
import nitime.utils as tsu
from nitime.viz import drawmatrix_channels

from IPython.core.display import display, HTML

"""
We then define a few parameters of the data: the TR and the bounds on the
frequency band of interest.
"""

TR = 2.00
f_ub = 0.15
f_lb = 0.02

path = './timeseries_CSVs'
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if 'ROI_Subject' in file:
            files.append(os.path.join(r, file))
X = np.array([np.zeros(435)]) #quite possibly unnecessarily complicated, unsure how to do this
names = []
for fnum in range(len(files)):
    if int(files[fnum][29:32]) in bids_subset_ids.conn_ID.values:
#         print(files[fnum][29:32])
        data_rec = np.genfromtxt(files[fnum], dtype=float, delimiter=',', names=True)

        roi_names = np.array(data_rec.dtype.names)
        nseq = len(roi_names)
        n_samples = data_rec.shape[0]
        data = np.zeros((nseq, n_samples))

        for n_idx, roi in enumerate(roi_names):
            data[n_idx] = data_rec[roi]
    #     display(data.shape)
    #     if np.isnan(data).any() or np.isinf(data).any():
    #         display(fnum)
        """
        We normalize the data in each of the ROIs to be in units of % change and
        initialize the TimeSeries object:
        """
        pdata = tsu.percent_change(data)
        time_series = ts.TimeSeries(pdata, sampling_interval=TR)
        """
        We initialize the GrangerAnalyzer object, while specifying the order of the
        autoregressive model to be 1 (predict the current behavior of the time-series
        based on one time-point back).
        """
        G = nta.GrangerAnalyzer(time_series, order=1)
        """
        We are only interested in the physiologically relevant frequency band
        (approximately 0.02 to 0.15 Hz).

        The spectral resolution is different in these two different analyzers. In the
        CoherenceAnalyzer, the spectral resolution depends on the size of the window
        used for calculating the spectral density and cross-spectrum, whereas in the
        GrangerAnalyzer it is derived, as determined by the user, from the MAR model
        used.

        For this reason, the indices used to access the relevant part of the spectrum
        will be different in the different analyzers.
        """
        freq_idx_G = np.where((G.frequencies > f_lb) * (G.frequencies < f_ub))[0]
        try:
            y_minus_x = np.mean(G.causality_xy[:, :, freq_idx_G] - G.causality_yx[:, :, freq_idx_G], -1)
            y_minus_x = y_minus_x[np.logical_not(np.isnan(y_minus_x))]
            temp = np.expand_dims(y_minus_x, axis=0)
            X = np.concatenate((X, temp), axis=0)
            display(bids_subset_ids.loc[bids_subset_ids['conn_ID'] == int(files[fnum][29:32])]['ID'].values[0])
            names += [bids_subset_ids.loc[bids_subset_ids['conn_ID'] == int(files[fnum][29:32])]['ID'].values[0]]
        except:
            print("an error occured that prevented causality calculation for", files[fnum])
X = X[1:]
# X = np.transpose(X)
display(X.shape)
causality_columns = []
for c in range(len(X[0])):
    causality_columns += ['causality_val_' + str(c)]
display(X)
causality_df = pd.DataFrame(X, columns=causality_columns)
causality_df['ID'] = names
display(causality_df)
causality_df.to_csv("granger.csv", index=False)
# need to put all causality values in a single array and save it to npy file
# probably need to make an empty array of the correct size first, as in read_matrices.ipynb

'A00028185'

'A00032875'

'A00033747'

'A00034854'

'A00035072'

...

'A00073600'

'A00073677'

(135, 435)

array([[-0.07734879, -0.02326116, -0.02560404, ...,  0.00767228,
        -0.04007535,  0.00413573],
       [ 0.08865912, -0.02261584,  0.07958646, ..., -0.02490081,
         0.02861762, -0.00086933],
       [-0.03111301,  0.02363727,  0.03266378, ..., -0.00263837,
        -0.07992016, -0.01957201],
       ...,
       [ 0.02504316, -0.01232973,  0.0596016 , ..., -0.099006  ,
        -0.02734357, -0.11738239],
       [ 0.06646207,  0.04105937,  0.10161895, ..., -0.00033607,
        -0.05217023, -0.00462329],
       [-0.06290784, -0.00859323,  0.04490803, ...,  0.00946496,
         0.01304842, -0.0092119 ]])

,causality_val_0,causality_val_1,causality_val_2,causality_val_3,causality_val_4,causality_val_5,causality_val_6,causality_val_7,causality_val_8,causality_val_9,...,causality_val_426,causality_val_427,causality_val_428,causality_val_429,causality_val_430,causality_val_431,causality_val_432,causality_val_433,causality_val_434,ID
0,-0.077349,-0.023261,-0.025604,-0.045863,0.026434,0.007452,-0.047753,-0.025576,-0.005988,0.022602,...,0.062982,0.089826,0.048796,0.005223,-0.005864,0.001145,0.007672,-0.040075,0.004136,A00028185
1,0.088659,-0.022616,0.079586,0.071261,0.050408,0.033138,0.021760,-0.020274,0.019489,0.023759,...,-0.008755,-0.026220,-0.001644,-0.006399,0.037191,0.050374,-0.024901,0.028618,-0.000869,A00032875
2,-0.031113,0.023637,0.032664,0.043367,-0.008001,0.091435,0.029972,-0.032429,0.032852,-0.007514,...,0.010078,-0.004546,-0.024402,0.029036,0.050541,-0.004396,-0.002638,-0.079920,-0.019572,A00033747
3,-0.004183,-0.012020,0.041849,0.092427,0.018554,0.020579,0.003571,0.052557,0.025421,0.008926,...,0.027165,0.002425,0.030094,0.025089,-0.001368,0.034693,-0.000109,0.007231,0.002459,A00034854
4,0.072271,0.137226,0.008788,0.015648,-0.026090,-0.005345,0.000281,0.144841,0.049128,-0.049018,...,-0.042693,-0.066591,-0.057090,-0.010328,-0.000321,0.007143,0.050529,-0.055297,-0.062330,A00035072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,-0.010537,-0.119205,0.164359,0.104824,0.030145,0.050524,0.080683,-0.114272,0.071177,0.017799,...,0.012113,-0.001119,-0.013332,0.011915,0.014101,0.001365,0.091827,0.271688,-0.080699,A00066827
131,0.089834,-0.211455,0.129028,0.131857,0.079990,0.143362,0.062040,-0.010379,0.124708,0.119515,...,-0.046567,-0.001347,-0.076139,0.016900,0.013151,-0.008062,-0.016252,-0.072384,-0.063142,A00066926
132,0.025043,-0.012330,0.059602,0.053493,0.027894,-0.008468,0.111642,0.005033,0.054174,0.005557,...,-0.012288,-0.001448,-0.013442,-0.036616,-0.029410,-0.016839,-0.099006,-0.027344,-0.117382,A00072203
133,0.066462,0.041059,0.101619,0.142323,0.304159,0.042322,0.066828,0.025737,0.126047,0.347869,...,0.043359,0.077157,0.019901,0.057452,0.095673,0.033148,-0.000336,-0.052170,-0.004623,A00073600
